In [2]:
import nltk
import sklearn_crfsuite
import sklearn
 
from sklearn.metrics import make_scorer
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import RandomizedSearchCV
 
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
 
 
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))[:1000]
#print 'train_sents : %s \n' % (train_sents[0])
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))[:100]
#print 'tests_sents : %s \n' % (test_sents[0])
#type(train_sents[0])



import sys
import time

from config import CONFIG

from tests import tests1, tests2

from normalizer import Normalizer

from lib.CharacterIndex import CharacterIndex
from lib.NaiveTokenizer import NaiveTokenizer
from lib.TextStreamer import TextStreamer
from lib.CONLL14ErrorCorrection import CONLL14ErrorCorrection
from lib.Parser import PatternParser
from lib.Report import Report

from lib.Tools import (
    FreqDist,
    splitter,
    strip_punct,
    tokenizer
)

from collections import (
    Counter,
    defaultdict as deft
)


def timestamp():
    return '.'.join([str(t) for t in time.localtime()[3:6]])


def get_name(template):
    i = 1
    while True:
        name = template % (timestamp(), i)
        if not os.path.exists(name):
            return name
        i += 1

PoS_l = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
         'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP',
         'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB']
PoS = {}
i = 1
for k in PoS_l:
	PoS[k] = i
	i += 1

# raw_input()

corpus = 'data/delorme.com_shu.pages_89.txt'

report = Report()

parser = PatternParser()

for C in CONFIG:
    
#     tests = tests1.items() + tests2.items()

    conll = CONLL14ErrorCorrection()
    
    tests = []
    for (left, err, right, corr, category), human  in conll:
        if err:
            test = (left, strip_punct(err).lower(), right,
                    strip_punct(corr).lower(), category, True)
        else:
            test = (left, strip_punct(corr), right, err, category, False)
        tests.append(test)
    tests = tests[:10000]

    targets = [test[1] for test in tests]

    #	Collect input from large text file:
    dump = []
    train_conll=[]
    for doc in TextStreamer(corpus, nb_sent=1000):
        for sent in splitter(doc):
            parse = parser(sent)
            for unit in parse.split():
                input_sent = []
                # print unit, '$ \n'
                for word_gr in unit :
                    word = word_gr[0]
                    tag = word_gr[1]
                    _tuple = (word,tag,word)
                    # print _tuple
                    input_sent.append(_tuple)
                # print input_sent    
                # raw_input()
            train_conll.append(input_sent)    
            #dump += [w.lower() for w in tokenizer(sent)]
    #freq_dist = Counter(dump + targets)
    print train_conll[0]


987it [00:06, 177.53it/s]


[(u'A', u'DT', u'A'), (u'bus', u'NN', u'bus'), (u'looms', u'VBZ', u'looms'), (u'out', u'IN', u'out'), (u'of', u'IN', u'of'), (u'the', u'DT', u'the'), (u'grey', u'JJ', u'grey'), (u'and', u'CC', u'and'), (u'blizzardy', u'JJ', u'blizzardy'), (u'conditions', u'NNS', u'conditions'), (u'and', u'CC', u'and'), (u'we', u'PRP', u'we'), (u'get', u'VB', u'get'), (u'on', u'IN', u'on'), (u'gratefully', u'RB', u'gratefully'), (u'.', u'.', u'.')]


989it [00:05, 188.14it/s]


[(u'A', u'DT', u'A'), (u'bus', u'NN', u'bus'), (u'looms', u'VBZ', u'looms'), (u'out', u'IN', u'out'), (u'of', u'IN', u'of'), (u'the', u'DT', u'the'), (u'grey', u'JJ', u'grey'), (u'and', u'CC', u'and'), (u'blizzardy', u'JJ', u'blizzardy'), (u'conditions', u'NNS', u'conditions'), (u'and', u'CC', u'and'), (u'we', u'PRP', u'we'), (u'get', u'VB', u'get'), (u'on', u'IN', u'on'), (u'gratefully', u'RB', u'gratefully'), (u'.', u'.', u'.')]


In [3]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
                '-1:word.lower()': word1.lower(),
                '-1:word.istitle()': word1.istitle(),
                '-1:word.isupper()': word1.isupper(),
                '-1:postag': postag1,
                '-1:postag[:2]': postag1[:2],
                })
    else:
        features['BOS'] = True
 
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
                '+1:word.lower()': word1.lower(),
                '+1:word.istitle()': word1.istitle(),
                '+1:word.isupper()': word1.isupper(),
                '+1:postag': postag1,
                '+1:postag[:2]': postag1[:2],
        }) 
    else:
        features['EOS'] = True
    return features
 
 
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
 
def sent2labels(sent):
    return [label for token, postag, label in sent]
 
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [6]:
print sent2features(train_sents[0])[0]
from tqdm import tqdm 

X_train = [sent2features(s) for s in train_conll[:100]]
#X_train = [sent2features(s) for s in train_sents]

y_train = [sent2labels(s) for s in train_conll[:100]]
#y_train = [sent2labels(s) for s in train_sents]
X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

print len(train_conll), train_conll[0]
print len(train_sents), train_sents[0]
 
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(tqdm(X_train), y_train)


print "alpha"
labels = list(crf.classes_)
# print labels
#labels.remove('O')
#labels = set(y_test)
print "alpha"

y_pred = crf.predict(tqdm(X_test))
# print(y_pred)

print metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

{'+1:word.isupper()': False, 'word.isupper()': False, 'BOS': True, 'word[-2:]': u'ne', '+1:postag': u'Fpa', 'word.isdigit()': False, 'postag': u'NP', 'bias': 1.0, 'postag[:2]': u'NP', '+1:word.lower()': u'(', '+1:word.istitle()': False, 'word.istitle()': True, 'word.lower()': u'melbourne', 'word[-3:]': u'rne', '+1:postag[:2]': u'Fp'}
1552 [(u'A', u'DT', u'A'), (u'bus', u'NN', u'bus'), (u'looms', u'VBZ', u'looms'), (u'out', u'IN', u'out'), (u'of', u'IN', u'of'), (u'the', u'DT', u'the'), (u'grey', u'JJ', u'grey'), (u'and', u'CC', u'and'), (u'blizzardy', u'JJ', u'blizzardy'), (u'conditions', u'NNS', u'conditions'), (u'and', u'CC', u'and'), (u'we', u'PRP', u'we'), (u'get', u'VB', u'get'), (u'on', u'IN', u'on'), (u'gratefully', u'RB', u'gratefully'), (u'.', u'.', u'.')]
1000 [(u'Melbourne', u'NP', u'B-LOC'), (u'(', u'Fpa', u'O'), (u'Australia', u'NP', u'B-LOC'), (u')', u'Fpt', u'O'), (u',', u'Fc', u'O'), (u'25', u'Z', u'O'), (u'may', u'NC', u'O'), (u'(', u'Fpa', u'O'), (u'EFE', u'NC', u'B-O

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 1428.57it/s]


alpha
alpha


 37%|███████████████▏                         | 37/100 [00:09<00:18,  3.33it/s]

ValueError: I/O operation on closed file